# Test Selenium 

https://selenium-python.readthedocs.io/getting-started.html

In [2]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def get_text_element_by_xpath(element, xpath, xpath2=None):
    try :
        return element.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        if xpath2 is None:
            return None
        try :
            return element.find_element(By.XPATH, xpath2).text
        except NoSuchElementException:
            return None
        
# plugin firefox pour récupérer les chemins XPATH: https://addons.mozilla.org/en-US/firefox/addon/xpath_finder/

In [21]:
# On crée une instance du web-driver Firefox (ou Chrome)
driver = webdriver.Firefox()
#driver = webdriver.Chrome("Chrome install path")  


driver.get("https://www.linkedin.com/in/ludovic-moncla/")
 
# En fonction de notre connection et des performances de notre machine il faudra attendre
# que la page charge avant de passer à la suite
#sleep(5)
item = {}
try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.TAG_NAME, "footer"))
    )

    # si la page se charge avec la popup de connexion, on la ferme
    driver.find_element(By.XPATH, "/html/body/div[4]/div/div/section/button").click()    
    try:
        # SUMMARY SECTION
        item['name'] = driver.find_element(By.TAG_NAME, 'h1').text
        item['description'] = driver.find_element(By.CLASS_NAME, "top-card-layout__headline").text

        item['location'] = get_text_element_by_xpath(driver, "//h3/div[@class='profile-info-subheader']/div[@class='not-first-middot'][1]/span", "//h3/div[@class='top-card__subline-item']")
        #item['location'] = driver.find_element(By.XPATH, "//h3/div[@class='profile-info-subheader']/div[@class='not-first-middot'][1]/span").text
                
        # EXPERIENCE SECTION 
        item['experience'] = []
        # Sélection de l'élément contenant les informations
        experience_list = driver.find_element(By.CLASS_NAME,'experience__list')

        # Extraction des informations pour chaque élément de la liste
        experience_items = experience_list.find_elements(By.CLASS_NAME,'experience-item')

        section_numbers = {}
        # get the section number for Expérience
        sections = driver.find_elements(By.XPATH,'/html/body/main/section[1]/div/section/section')
        for i, section in enumerate(sections):
            print(i, section)
            if section.find_element(By.TAG_NAME, 'h2').text == 'Expérience': # Maybe need to translate if the brwoser is not in french
                print('Expérience section found')
                section_numbers['experience'] = str(i+1)
            if section.find_element(By.TAG_NAME, 'h2').text == 'Formation':
                print('Formation section found')
                section_numbers['formation'] = str(i+1)
        


        for i, experience_item in enumerate(experience_items):
            # Création d'un dictionnaire pour stocker les informations
            experience_data = {}
            
            '''
            # Extraction des informations
            experience_data['Titre'] = experience_list.find_element(By.XPATH,"//div/ul[@class='experience__list']/li[1]/div/h3/span[@class='experience-item__title']").text
            experience_data['Nom de lentreprise'] = experience_list.find_element(By.CLASS_NAME,'profile-section-card__subtitle-link').text
            experience_data['Durée'] = experience_list.find_element(By.CLASS_NAME,'date-range').text
            experience_data['Lieu'] = experience_list.find_element(By.CLASS_NAME,'experience-item__location').text
            experience_data['Description'] = experience_list.find_element(By.CLASS_NAME,'show-more-less-text__text--less').text
            '''

            # LM: il faut utiliser experience_item et pas experience_list comme on est dans la boucle
            # il faut lever les exceptions pour les elements qui n'existent pas 
            #experience_data['titre'] = get_text_element_by_xpath(experience_item, "//div[@class='profile-section-card__contents']/h3[@class='profile-section-card__title']", "//div/h3/span[@class='experience-item__title']")
            #experience_data['nom_entreprise'] = get_text_element_by_xpath(experience_item, "//div[@class='profile-section-card__contents']/h4[@class='profile-section-card__subtitle']", "//div/h4/span[@class='experience-item__subtitle']")
            #experience_data['duree'] = get_text_element_by_xpath(experience_item, "//div/div/p/span[@class='date-range']/span[@class='before:middot']")
            #experience_data['lieu'] = get_text_element_by_xpath(experience_item, "//p[@class='experience-item__location']", "//div/div/p[@class='experience-item__meta-item']")
            #experience_data['description'] = get_text_element_by_xpath(experience_item, "//div/p[@class='show-more-less-text__text--less']")
            
            experience_data['titre'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['experience']+"]/div/ul/li["+str(i+1)+"]/div/h3/span")
            experience_data['nom_entreprise'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['experience']+"]/div/ul/li["+str(i+1)+"]/div/h4/a/span")
            experience_data['duree'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['experience']+"]/div/ul/li["+str(i+1)+"]/div/div/p[1]/span/span")
            experience_data['lieu'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['experience']+"]/div/ul/li["+str(i+1)+"]/div/div/p[2]")
            experience_data['description'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['experience']+"]/div/ul/li["+str(i+1)+"]/div/div/div/div/p")
            
            
            item['experience'].append(experience_data)
            
            
        # EDUCATION SECTION
        item['education']=[]
        # Sélection de l'élément contenant les informations
        education_list = driver.find_element(By.CLASS_NAME,'education__list')

        # Extraction des informations pour chaque élément de la liste
        education_items = education_list.find_elements(By.CLASS_NAME,'education__list-item')

        


        for i, education_item in enumerate(education_items):
            education_data = {}
            
            #education_data['Nom établissement'] = education_item.find_element(By.CLASS_NAME,'profile-section-card__title-link').text
            #education_data['Diplôme'] = education_item.find_element(By.CLASS_NAME,'education__item--degree-info').text
            #education_data['Durée'] = education_item.find_element(By.CLASS_NAME,'date-range').text

            #education_data['nom_etablissement'] = get_text_element_by_xpath(education_item, "//div[@class='profile-section-card__contents']/h3[@class='profile-section-card__title']/a", "/div/h3/a")
            #education_data['diplome'] = get_text_element_by_xpath(education_item, "/div[@class='profile-section-card__contents']/h4[@class='profile-section-card__subtitle']/span[@class='education__item--degree-info']", "/div/h4/span")
            #education_data['duree'] = get_text_element_by_xpath(education_item, "//div[@class='profile-section-card__contents']/div[@class='profile-section-card__meta']/p/span[@class='date-range']/time", "/div/div/p/span/time")
            
            education_data['nom_etablissement'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['formation']+"]/div/ul/li["+str(i+1)+"]/div/h3/a")
            education_data['diplome'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['formation']+"]/div/ul/li["+str(i+1)+"]/div/h4")
            education_data['duree'] = get_text_element_by_xpath(driver, "/html/body/main/section[1]/div/section/section["+section_numbers['formation']+"]/div/ul/li["+str(i+1)+"]/div/div/p/span")
            

            item['education'].append(education_data)

        # Affichage de la liste de dictionnaires
        #for i, education_data in enumerate(education_data_list, start=1):
            #print(f"Éducation #{i}:")
            #print(education_data)
            #print()

        #driver.quit()
    except NoSuchElementException:
        pass
    
except NoSuchElementException:
    print('Error: The page requests login! ')
finally:
    pass
    #driver.quit()

item

0 <selenium.webdriver.remote.webelement.WebElement (session="4592cc24-9f63-4043-807c-f4a862193471", element="ec03bc6b-7073-43f3-9cb9-173e2385941d")>
1 <selenium.webdriver.remote.webelement.WebElement (session="4592cc24-9f63-4043-807c-f4a862193471", element="bf53a7ad-3c2d-499f-8b75-97c59a20f6e6")>
2 <selenium.webdriver.remote.webelement.WebElement (session="4592cc24-9f63-4043-807c-f4a862193471", element="1703952a-1080-4937-afde-ba9169ead905")>
Expérience section found
3 <selenium.webdriver.remote.webelement.WebElement (session="4592cc24-9f63-4043-807c-f4a862193471", element="4d36f01d-f73f-415c-86ca-5b53be09d5cc")>
Formation section found
4 <selenium.webdriver.remote.webelement.WebElement (session="4592cc24-9f63-4043-807c-f4a862193471", element="55adcf3a-4606-401f-9dde-58e9d712bbd1")>
5 <selenium.webdriver.remote.webelement.WebElement (session="4592cc24-9f63-4043-807c-f4a862193471", element="1ee40f5f-be2c-4bee-ac8f-fa0d24f48464")>


{'name': 'Ludovic Moncla',
 'description': 'Associate Professor, INSA Lyon, LIRIS CNRS UMR 5205',
 'location': 'Villeurbanne, Auvergne-Rhône-Alpes, France',
 'experience': [{'titre': 'Associate Professor',
   'nom_entreprise': 'INSA Lyon - Institut National des Sciences Appliquées de Lyon',
   'duree': '5 ans 3 mois',
   'lieu': 'Région de Lyon, France',
   'description': None},
  {'titre': 'Teaching and Research Assistant',
   'nom_entreprise': "Institut de Recherche de l'Ecole Navale",
   'duree': '2 ans',
   'lieu': 'Lanvéoc-Poulmic',
   'description': 'My current research activity focus on Natural Language Processing (NLP), Information Extraction (IE) and Geographic Information Systems (GIS).'},
  {'titre': None,
   'nom_entreprise': None,
   'duree': None,
   'lieu': None,
   'description': None},
  {'titre': 'PhD in Computer science',
   'nom_entreprise': 'Universidad de Zaragoza',
   'duree': '3 ans 2 mois',
   'lieu': 'Zaragoza, Espagne',
   'description': "PhD :\nTitle : Autom

* see: https://selenium-python.readthedocs.io/locating-elements.html